In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
#import tensorflow as tf

# Import our input dataset
#data = Path('./Data/FIMA_cleaned.csv')
data = Path('./Data/FEMA_Clean14.csv')
FEMA_df = pd.read_csv(data)
FEMA_df.head()

C:\Users\patel\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
0,OCEANSIDE,1998-02-07,999.0,X,33.2,-117.3,4.0,1.0,1963-01-01,0.00,CA,92056
1,NEW ORLEANS,2005-08-29,999.0,X,29.9,-90.0,2.0,1.0,1967-07-01,0.00,LA,70131
2,NAVARRE,1998-09-28,999.0,X,30.4,-86.9,1.0,1.0,1972-01-01,8813.21,FL,32566
3,BEAUFORT,1994-10-07,999.0,X,32.4,-80.7,2.0,1.0,1960-01-01,2906.00,SC,29902
4,MELBOURNE,1996-03-11,999.0,X,28.3,-80.7,1.0,1.0,1988-01-01,3875.53,FL,32940


In [26]:
#

FEMA_df.dtypes

reportedcity                           object
dateofloss                             object
elevationdifference                   float64
floodzone                              object
latitude                              float64
longitude                             float64
numberoffloorsintheinsuredbuilding    float64
occupancytype                         float64
originalconstructiondate               object
amountpaidonbuildingclaim             float64
state                                  object
reportedzipcode                        object
dtype: object

In [29]:
FEMA_df['originalconstructiondate'] =  pd.to_datetime(FEMA_df['originalconstructiondate'], format='%Y-%m-%d', errors='ignore')
FEMA_df['dateofloss'] =  pd.to_datetime(FEMA_df['dateofloss'], format='%Y-%m-%d', errors='ignore')
FEMA_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

In [30]:
FEMA_df[['lossyear','lossmonth','lossday']] = FEMA_df.dateofloss.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

In [34]:
FEMA_df[['builtyear','builtmonth','builtlossday']] = FEMA_df.originalconstructiondate.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))

In [35]:
FEMA_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
lossyear                                      object
lossmonth                                     object
lossday                                       object
builtyear                                     object
builtmonth                                    object
builtlossday                                  object
dtype: object

In [39]:
#convert all year month dates to floats

FEMA_df['lossyear'] = FEMA_df['lossyear'].astype(float)
FEMA_df['lossmonth'] = FEMA_df['lossmonth'].astype(float)
FEMA_df['lossday'] = FEMA_df['lossday'].astype(float)
FEMA_df['builtyear'] = FEMA_df['builtyear'].astype(float)
FEMA_df['builtmonth'] = FEMA_df['builtmonth'].astype(float)
FEMA_df['builtlossday'] = FEMA_df['builtlossday'].astype(float)

In [40]:
FEMA_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
lossyear                                     float64
lossmonth                                    float64
lossday                                      float64
builtyear                                    float64
builtmonth                                   float64
builtlossday                                 float64
dtype: object

In [41]:
#age of the property at the time of loss

FEMA_df['propertyage'] = FEMA_df['lossyear'] - FEMA_df['builtyear'] 

In [43]:
FEMA_df.iloc[0]

reportedcity                                    OCEANSIDE
dateofloss                            1998-02-07 00:00:00
elevationdifference                                   999
floodzone                                               X
latitude                                             33.2
longitude                                          -117.3
numberoffloorsintheinsuredbuilding                      4
occupancytype                                           1
originalconstructiondate              1963-01-01 00:00:00
amountpaidonbuildingclaim                               0
state                                                  CA
reportedzipcode                                     92056
lossyear                                             1998
lossmonth                                               2
lossday                                                 7
builtyear                                            1963
builtmonth                                              1
builtlossday  

In [52]:
FEMA_df.to_csv('.\Data\FEMA_datesplit.csv', index = False)

In [84]:
FEMA_df['location'] = FEMA_df['reportedcity'].str.cat(FEMA_df['state'],sep=", ")

In [92]:
locations = FEMA_df.location.unique()
locations.shape

(45713,)

In [88]:
df = FEMA_df.copy()
df = df.drop("reportedzipcode", axis=1)
df = df.drop("state", axis=1)
#df = df.drop("floodzone", axis=1)
df = df.drop("dateofloss", axis=1)
df = df.drop("originalconstructiondate", axis=1)
df = df.drop("reportedcity", axis=1)
df = df.drop("elevationdifference", axis=1)
#df = df.drop("latitude", axis=1)
#df = df.drop("longitude", axis=1)
#df = df.drop("lossday", axis=1)
#df = df.drop("builtlossday", axis=1)
#df = df.drop("propertyage", axis=1)
FEMA_encoded = pd.get_dummies(df, columns=["location", "floodzone", "occupancytype", "numberoffloorsintheinsuredbuilding"])#, "reportedzipcode"])


y =FEMA_encoded.copy()
X = FEMA_encoded.copy()
X = X.drop("amountpaidonbuildingclaim", axis=1)

MemoryError: Unable to allocate 84.6 GiB for an array with shape (1986350, 45713) and data type uint8

In [86]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y['amountpaidonbuildingclaim'],
                                                   random_state=1)

In [87]:
# Train the Classifier

from sklearn.linear_model import LinearRegression

#random forest, neural networks with linear regression
model = LinearRegression() 
model.fit(X_train, y_train, sample_weight=43)

ValueError: could not convert string to float: 'SLIDELL, LA'

In [ ]:
score_with_location= model.score(X_test, y_test)
score_with_location

In [83]:
score_without_elevationdiff = model.score(X_test, y_test)
score_without_elevationdiff

0.13321762639490686

In [78]:
score_with_encoded_floors = model.score(X_test, y_test)
score_with_encoded_floors

0.13331513704442144

In [72]:
score_with_encoded_ocptype = model.score(X_test, y_test)
score_with_encoded_ocptype

0.13256500646283842

In [68]:
score_without_days = model.score(X_test, y_test)
score_without_days

0.11738635626723981

In [57]:
score_without_zipcodes = model.score(X_test, y_test)

In [63]:
score_without_propertyage = model.score(X_test, y_test)

In [64]:
score_without_propertyage

0.12948079205780605

In [58]:
score_without_zipcodes

0.1294807920578066

In [59]:
FEMA_df.reportedcity[0]

'OCEANSIDE'

In [ ]:
#FEMA_2019['location'] = FEMA_2019['reportedcity'].str.cat(FEMA_2019['state'],sep=", ")

In [7]:
Floodzones = FEMA_df.floodzone.unique()
Floodzones

array(['X', 'AE', 'VE', 'A00', 'A10', 'A08', 'V', 'C', 'A', 'A14', 'AO',
       'B', 'A15', 'A99', 'A11', 'A0B', 'A13', 'AOB', 'V12', 'A06', 'AHB',
       'D', 'A12', 'A18', 'V21', 'V14', 'A05', 'A16', 'AH', 'A03', 'V18',
       'V07', 'A04', 'V16', 'V15', 'A07', 'A01', 'A09', 'V11', 'A02',
       'AR', 'V13', 'V17', 'V20', 'V10', 'V06', 'V05', 'V09', 'V08',
       'V03', 'A17', 'A21', 'A20', 'A22', 'A19', 'A24', 'A30', 'V19',
       'A25', 'V22', 'V04', 'V02', 'A23', 'V01', 'V23', 'A26', 'A28',
       'A27', 'A29', 'V24', 'V30', 'V27', 'AA', '000', 'OOO', 'A--',
       'AO5', 'A4'], dtype=object)

In [11]:
FEMA_df.floodzone.nunique(dropna = True) 

78

In [20]:
FEMA_df.floodzone[0] == Floodzones[0]

True

In [23]:

#'FEMA_'&Floodzones[0].append(FEMA_df.iloc[0])


In [22]:
#for i in FEMA_df.floodzone:
   # for x in Floodzones:
       # if FEMA_df.floodzone[i] == Floodzones[x]:
            #'FEMA_'&Floodzones[x].append(FEMA_df.iloc[i])